In [1]:
# === P-TLI — Threshold por monto (Inbound Cash) ================================
# Regla: When an Inbound Cash transaction is over [Amount] {var.Amount} CLP, apply {action}.
# Justificación: seleccionar el monto en el percentil 95 de la distribución.

import pandas as pd
import numpy as np

# -------- Parámetros editables --------
PATH = "../../data/tx/datos_trx__with_subsub.csv"
SUBSUBSEGMENTS = "R-Low"         # <-- ajusta el sub-subsegmento
PCTS = [90, 95, 97, 99]         # percentiles a reportar

# -------- Carga csv --------
df = pd.read_csv(PATH, dtype={"customer_id": "string"}, encoding="utf-8-sig")

# Filtrado por sub-subsegmento
if isinstance(SUBSUBSEGMENTS, str):
    target_labels = {SUBSUBSEGMENTS}
else:
    target_labels = set(map(str, SUBSUBSEGMENTS))   

df = df[df["customer_sub_sub_type"].astype(str).isin(target_labels)].copy()

# -------- Filtro según regla --------
df["tx_base_amount"] = pd.to_numeric(df["tx_base_amount"], errors="coerce")
mask = (
    (df["tx_direction"].astype(str).str.title() == "Inbound") &
    (df["tx_type"].astype(str).str.title() == "Cash") &
    (df["tx_base_amount"] > 0)  # montos válidos y positivos en CLP
)
g = df.loc[mask, ["tx_base_amount"]].dropna()

if g.empty:
    print("No hay transacciones elegibles para P-TLI con los filtros dados.")
else:
    s = g["tx_base_amount"].astype(float)
    stats = {f"p{p}": float(np.percentile(s, p)) for p in PCTS}

    # Recomendado:p95
    recommended_amount = int(round(stats["p95"]))

    print("=== P-TLI — Percentiles de monto (CLP, Inbound Cash) ===")
    for p in PCTS:
        v = stats[f"p{p}"]
        print(f"p{p:>2}: {v:,.0f}")
    print(f"\nAmount recomendado (p95): {recommended_amount:,.0f} CLP")


C:\Users\vinso\AppData\Local\Temp\ipykernel_8636\1733011156.py:14: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH, dtype={"customer_id": "string"}, encoding="utf-8-sig")


=== P-TLI — Percentiles de monto (CLP, Inbound Cash) ===
p90: 12,357,278
p95: 33,000,000
p97: 56,149,679
p99: 130,000,000

Amount recomendado (p95): 33,000,000 CLP


# Simulación alertas

In [11]:
# === P-TLI — Matching (Actual vs Reales) + Escenarios (subsub) ==================
# Lógica simulación:
#   tx_direction = Inbound
#   AND tx_base_amount > [Amount]
#   AND tx_date_time >= 2025-03-01 (inclusive)
#
# "Actual" se corre con customer_sub_type == SUBSEGMENT_REAL (p.ej., Retail)
# Escenarios (p95/p97/p99/...) se corren con customer_sub_sub_type in SUBSUBSEGMENTS_NEW (p.ej., ["R-Low"])

import pandas as pd
import numpy as np
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

# ---------------------------------------------------------------------
# Paths, parámetros y segmentos
# ---------------------------------------------------------------------
TX_PATH      = "../../data/tx/datos_trx__with_subsub_oficial.csv"
ALERTS_PATH  = "../../data/alertas marzo-agosto 2025 enriched.csv"

# Para comparación con reales y "Actual"
SUBSEGMENT_REAL      = "Retail"       # customer_sub_type

# Para simulación de escenarios nuevos (percentiles)
SUBSUBSEGMENTS_NEW   = ["IV-1"]      # customer_sub_sub_type

# Dirección (TLI = Inbound)
DIR = "Inbound"

# Parámetros (puedes agregar más escenarios libremente)
PARAMS = {
    "Actual": {"Amount": 299_000_000},
    "p95":    {"Amount": 491491500},
    "p97":    {"Amount": 756750796},
    "p99":    {"Amount": 1500000000},
}

COUNT_FROM = pd.Timestamp("2025-02-21", tz="UTC")
COUNT_TO   = None  # p.ej. pd.Timestamp("2025-08-31 23:59:59", tz="UTC")

# ---------------------------------------------------------------------
# Helpers
# ---------------------------------------------------------------------
def _to_dt_utc(s):
    return pd.to_datetime(s, errors="coerce", utc=True)

def _build_key(df, prefer_tx_id=True):
    has_txid = ("tx_id" in df.columns)
    if prefer_tx_id and has_txid:
        key = df["tx_id"].astype("string").str.strip()
        if key.notna().sum() == 0:
            has_txid = False
    if not prefer_tx_id or not has_txid:
        cust = df.get("customer_id", "").astype("string").fillna("").str.strip()
        ts   = _to_dt_utc(df.get("tx_date_time")).dt.strftime("%Y-%m-%dT%H:%M:%SZ")
        amt  = pd.to_numeric(df.get("tx_base_amount"), errors="coerce").round(0).fillna(-1).astype("Int64").astype(str)
        d    = df.get("tx_direction", "").astype(str).str.title()
        key  = cust.str.cat([ts, amt, d], sep="|")
    return key

def _load_tx(path):
    tx = pd.read_csv(path, dtype={"customer_id":"string"}, encoding="utf-8-sig", low_memory=False)
    tx["customer_sub_type"]      = tx.get("customer_sub_type","").astype(str)
    tx["customer_sub_sub_type"]  = tx.get("customer_sub_sub_type","").astype(str)
    tx["tx_direction"]   = tx.get("tx_direction","").astype(str).str.title()
    tx["tx_base_amount"] = pd.to_numeric(tx.get("tx_base_amount"), errors="coerce")
    tx["tx_date_time"]   = _to_dt_utc(tx.get("tx_date_time"))
    return tx

def _load_reales_ptli(path):
    dtypes = {"alert_id":"string","rule_code":"string","customer_id":"string","tx_id":"string"}
    real = pd.read_csv(path, dtype=dtypes, encoding="utf-8-sig", low_memory=False)
    real["rule_code"]        = real["rule_code"].astype(str).str.strip()
    real["customer_sub_type"]= real.get("customer_sub_type","").astype(str)
    real["tx_direction"]     = real.get("tx_direction","").astype(str).str.title()
    real["tx_base_amount"]   = pd.to_numeric(real.get("tx_base_amount"), errors="coerce")
    real["tx_date_time"]     = _to_dt_utc(real.get("tx_date_time"))
    return real

def _filter_dates(df, col="tx_date_time", start=COUNT_FROM, end=COUNT_TO):
    m = df[col] >= start
    if end is not None:
        m &= (df[col] <= end)
    return df.loc[m].copy()

# ---------------------------------------------------------------------
# 1) SIMULACIÓN "Actual" (segmento: customer_sub_type == SUBSEGMENT_REAL)
# ---------------------------------------------------------------------
tx_all = _load_tx(TX_PATH)

tx_actual = tx_all[tx_all["customer_sub_type"] == SUBSEGMENT_REAL].copy()
tx_actual = _filter_dates(tx_actual, "tx_date_time", COUNT_FROM, COUNT_TO)

g_act = tx_actual[(tx_actual["tx_direction"].eq(DIR)) & tx_actual["tx_base_amount"].notna()].copy()
A_act = float(PARAMS["Actual"]["Amount"])
sim_hits_actual = g_act[g_act["tx_base_amount"] > A_act].copy()
sim_hits_actual["match_key"] = _build_key(sim_hits_actual, prefer_tx_id=True)

# ---------------------------------------------------------------------
# 2) ALERTAS REALES (rule_code == P-TLI; customer_sub_type == SUBSEGMENT_REAL)
# ---------------------------------------------------------------------
real = _load_reales_ptli(ALERTS_PATH)
real = real[(real["rule_code"] == "P-TLI") & (real["customer_sub_type"] == SUBSEGMENT_REAL)].copy()
real = real[(real["tx_date_time"].isna()) | (real["tx_date_time"] >= COUNT_FROM)]
if COUNT_TO is not None:
    real = real[(real["tx_date_time"].isna()) | (real["tx_date_time"] <= COUNT_TO)]
real["match_key"] = _build_key(real, prefer_tx_id=True)

# Matching
sim_keys  = set(sim_hits_actual["match_key"].dropna().unique())
real_keys = set(real["match_key"].dropna().unique())
reales_sin_sim = real[~real["match_key"].isin(sim_keys)].copy()
sim_sin_reales = sim_hits_actual[~sim_hits_actual["match_key"].isin(real_keys)].copy()

# Logs claros
print("=== P-TLI — Resumen matching (Actual vs Reales) ===")
print(f"Alertas reales: {len(real):,}")
print(f"Alertas simuladas: {len(sim_hits_actual):,}")
print(f"Alertas reales que no están en la simulación: {len(reales_sin_sim):,}")
print(f"Alertas en la simulación que no saltaron realmente: {len(sim_sin_reales):,}")

print("\n--- Reales sin simulación (head) ---")
display(reales_sin_sim[["alert_id","rule_code","customer_id","tx_id","tx_date_time","tx_direction","tx_base_amount","match_key"]].head(20))

print("\n--- Simulación sin reales (head) ---")
display(sim_sin_reales[["customer_id","tx_id","tx_date_time","tx_direction","tx_base_amount","match_key"]].head(20))

# ---------------------------------------------------------------------
# 3) ESCENARIOS (percentiles) en subsubsegmentos (customer_sub_sub_type in SUBSUBSEGMENTS_NEW)
# ---------------------------------------------------------------------
tx_new = tx_all[tx_all["customer_sub_sub_type"].isin(SUBSUBSEGMENTS_NEW)].copy()
tx_new = _filter_dates(tx_new, "tx_date_time", COUNT_FROM, COUNT_TO)
g_new  = tx_new[(tx_new["tx_direction"].eq(DIR)) & tx_new["tx_base_amount"].notna()].copy()

scenario_counts = {}
for esc, pars in PARAMS.items():
    amt = float(pars["Amount"])
    scenario_counts[esc] = int((g_new["tx_base_amount"] > amt).sum())

print("\n=== P-TLI — Escenarios (subsub) ===")
display(pd.DataFrame([scenario_counts]))


=== P-TLI — Resumen matching (Actual vs Reales) ===
Alertas reales: 190
Alertas simuladas: 120
Alertas reales que no están en la simulación: 70
Alertas en la simulación que no saltaron realmente: 0

--- Reales sin simulación (head) ---


,alert_id,rule_code,customer_id,tx_id,tx_date_time,tx_direction,tx_base_amount,match_key
823,82123,P-TLI,15099381,201934025.0,2025-02-27 00:00:00+00:00,Inbound,"279,608,544",201934025.0
827,82127,P-TLI,18638500,201934221.0,2025-02-27 00:00:00+00:00,Inbound,"279,608,544",201934221.0
831,82131,P-TLI,13332973,201935224.0,2025-02-27 00:00:00+00:00,Inbound,"279,608,544",201935224.0
835,82135,P-TLI,13851375,201933739.0,2025-02-27 00:00:00+00:00,Inbound,"279,608,544",201933739.0
839,82139,P-TLI,16612187,201933744.0,2025-02-27 00:00:00+00:00,Inbound,"279,608,544",201933744.0
948,82291,P-TLI,18638500,201955408.0,2025-02-27 00:00:00+00:00,Inbound,"279,608,544",201955408.0
1851,84874,P-TLI,6374943,202454452.0,2025-03-05 00:00:00+00:00,Inbound,"250,000,000",202454452.0
2134,85683,P-TLI,15104845,202526503.0,2025-03-06 00:00:00+00:00,Inbound,"260,000,000",202526503.0
2212,85976,P-TLI,6370180,202585750.0,2025-03-07 00:00:00+00:00,Inbound,"232,470,000",202585750.0
2271,86045,P-TLI,26239495,202635350.0,2025-03-07 00:00:00+00:00,Inbound,"228,780,000",202635350.0



--- Simulación sin reales (head) ---


,customer_id,tx_id,tx_date_time,tx_direction,tx_base_amount,match_key



=== P-TLI — Escenarios (subsub) ===


,Actual,p95,p97,p99
0,588,339,213,66
